In [1]:
# Import Dependencies
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from keras.models import Sequential, Model, model_from_json
from keras.layers import Input
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from scipy import ndimage

Using plaidml.keras.backend backend.


In [2]:
def binary_model(img_dims, classes):
    # Initialize the model
    model = Sequential()
    input_shape = (img_dims[1], 
                   img_dims[0], 
                   img_dims[2])
    chan_dim = -1
    
    # Update input_shape if using channels_first
    if K.image_data_format() == "channels_first":
        input_shape = (img_dims[2], 
                       img_dims[1], 
                       img_dims[0])
        chan_dim = 1

    model.add(ZeroPadding2D(padding=(1, 1), input_shape=input_shape))
    model.add(Conv2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(4096, 7, 7, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Conv2D(4096, 1, 1, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Conv2D(classes, 1, 1, name='fc8'))
    model.add(Flatten())
    model.add(Activation('softmax'))

    # return the constructed network architecture
    return model
    

In [3]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os

In [4]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMG_DIMS = (224, 224, 3)

MODEL_NAME = "gender"

data_set_dir = 'data/dataset/' + MODEL_NAME


In [ ]:
def split_train_test_set(directory):
    
    classifiers = next(os.walk(data_set_dir))[1]
    
    print("[INFO] loading images from {}".format(directory))
    # initialize the data and labels
    data = []
    labels = []
    
    # grab the image paths and randomly shuffle them
    image_paths = sorted(list(paths.list_images(directory)))
    random.seed(42)
    random.shuffle(image_paths)
    
    # loop over the input images
    for image_path in image_paths:
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(image_path)
        image = cv2.resize(image, (IMG_DIMS[0], IMG_DIMS[1]))
        image = img_to_array(image)
        data.append(image)
 
        # extract the class label from the image path and update the
        # labels list        
        label = image_path.split(os.path.sep)[-2]
        l = classifiers.index(label)
        labels.append(l)
        
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
     
    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    train_X, test_X, train_Y, test_Y = train_test_split(data,
                                                        labels,
                                                        test_size=0.2,
                                                        random_state=42)
 
    # convert the labels from integers to vectors
    train_Y = to_categorical(train_Y, num_classes=2)
    test_Y = to_categorical(test_Y, num_classes=2)
    
    return train_X, test_X, train_Y, test_Y, classifiers
        

In [ ]:
def show_model_performance_graph(model):
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = EPOCHS
    plt.plot(np.arange(0, N), 
             model.history["loss"], 
             label="train_loss")
    plt.plot(np.arange(0, N), 
             model.history["val_loss"], 
             label="val_loss")
    
    plt.plot(np.arange(0, N), 
             model.history["acc"], 
             label="train_acc")
    plt.plot(np.arange(0, N), 
             model.history["val_acc"], 
             label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.show()

In [ ]:
def train_model(dataset_dir):
    train_X, test_X, train_Y, test_Y, classifiers = split_train_test_set(dataset_dir)
    
    # initialize the model
    print("[INFO] compiling model...")
    model = binary_model(IMG_DIMS, classes=2)
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt, 
                  metrics=["accuracy"])
    
    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=30, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode="nearest")
    
    # train the network
    print("[INFO] training network...")
    h = model.fit_generator(
        aug.flow(train_X, train_Y, batch_size=BS),
        validation_data=(test_X, test_Y), 
        steps_per_epoch=len(train_X) // BS,
        epochs=EPOCHS, 
        verbose=1)
    # save the model to disk
    print("[INFO] serializing network...")
    model.save(MODEL_NAME + "_model.h5")
    return h, model, classifiers

In [ ]:
h, model, classifiers = train_model(data_set_dir)

[INFO] loading images from data/dataset/gender


In [ ]:
show_model_performance_graph(h)

### Test

In [ ]:
# model = binary_model(IMG_DIMS, classes=2)
# model.load_weights("./gender_model.h5")

In [ ]:
# load the image
image = cv2.imread('./test5.jpg')
orig = image.copy()

# pre-process the image for classification
image = cv2.resize(image, (IMG_DIMS[0], IMG_DIMS[1]))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [ ]:
prediction = model.predict(image)[0]

In [ ]:
for i, label in enumerate(classifiers):
    print("{}: {:.2}%".format(label, prediction[i]))